This jupyter notebook conducts experiments to see what are the best parameters and filters to use to shrink the transaction graph.
Some parameters to consider:
1. The timesteps on which we decide which the wallets to keep
2. The number of transactions conducted by each wallet
3. The connectivity of wallet
4. The amount of ethereum transacted from/to the wallet

# 1. Get a basic sense: check graph size of one time step

In [1]:
from graph_tools.components.graph import TransactionGraph, GraphGenerator

In [ ]:
short_period_graph = TransactionGraph(transaction_file='data/transaction table 20190101-20190401.csv', total_time_steps=3)

 67%|███████████████████████████████████████████▎                     | 2/3 [01:19<00:39, 39.79s/it]